# 機器學習模型建立 - 三高預測

**目標**: 使用處理好的資料建立預測模型

**策略**:
- 使用 T1 + T2 的資料訓練模型
- 預測 T3 時的三高狀態
- 從簡單的 Logistic Regression 開始

**日期**: 2025-11-18

## 1. 載入套件與資料

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# 機器學習套件
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix, classification_report
import warnings
warnings.filterwarnings('ignore')

# 設定中文字型
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei', 'Microsoft YaHei', 'SimHei', 'Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

print("套件載入成功！")

In [ ]:
# 載入處理好的資料
data_path = Path('../../data/processed/SUA_CVDs_wide_format.csv')
df = pd.read_csv(data_path)

print(f"✅ 資料載入成功！")
print(f"資料形狀: {df.shape[0]:,} 人, {df.shape[1]} 個欄位")
print(f"\n欄位預覽:")
print(df.columns[:10].tolist())

## 2. 準備特徵和目標變數

我們將使用 **T1 和 T2 的資料** 來預測 **T3 的三高狀態**

In [ ]:
# 定義特徵欄位（使用 T1 和 T2 的資料）
# 基礎特徵
t1_features = ['FBG_T1', 'TC_T1', 'Cr_T1', 'UA_T1', 'GFR_T1', 'BMI_T1', 'SBP_T1', 'DBP_T1']
t2_features = ['FBG_T2', 'TC_T2', 'Cr_T2', 'UA_T2', 'GFR_T2', 'BMI_T2', 'SBP_T2', 'DBP_T2']

# Δ 變化特徵（T2 - T1）
delta1_features = ['Delta1_FBG', 'Delta1_TC', 'Delta1_Cr', 'Delta1_UA', 
                   'Delta1_GFR', 'Delta1_BMI', 'Delta1_SBP', 'Delta1_DBP']

# 人口統計特徵
demographic_features = ['sex', 'Age']

# 組合所有特徵
all_features = demographic_features + t1_features + t2_features + delta1_features

print(f"特徵總數: {len(all_features)} 個")
print(f"  - 人口統計: {len(demographic_features)} 個")
print(f"  - T1 特徵: {len(t1_features)} 個")
print(f"  - T2 特徵: {len(t2_features)} 個")
print(f"  - Δ1 特徵: {len(delta1_features)} 個")

In [ ]:
# 定義目標變數（T3 時的三高狀態）
target_columns = ['hypertension_T3', 'hyperglycemia_T3', 'dyslipidemia_T3']

# 準備 X（特徵）和 y（目標）
X = df[all_features]
y_hypertension = df['hypertension_T3']
y_hyperglycemia = df['hyperglycemia_T3']
y_dyslipidemia = df['dyslipidemia_T3']

print("目標變數分佈:")
print(f"\n高血壓 (hypertension_T3):")
print(y_hypertension.value_counts())
print(f"  患病率: {(y_hypertension == 2).mean():.2%}")

print(f"\n高血糖 (hyperglycemia_T3):")
print(y_hyperglycemia.value_counts())
print(f"  患病率: {(y_hyperglycemia == 2).mean():.2%}")

print(f"\n高血脂 (dyslipidemia_T3):")
print(y_dyslipidemia.value_counts())
print(f"  患病率: {(y_dyslipidemia == 2).mean():.2%}")

## 3. 資料分割與標準化

In [ ]:
# 將目標變數轉換為二元分類（1=正常, 2=患病 → 0=正常, 1=患病）
y_hypertension_binary = (y_hypertension == 2).astype(int)
y_hyperglycemia_binary = (y_hyperglycemia == 2).astype(int)
y_dyslipidemia_binary = (y_dyslipidemia == 2).astype(int)

# 分割訓練集和測試集（80% 訓練, 20% 測試）
X_train, X_test, y_train_hp, y_test_hp = train_test_split(
    X, y_hypertension_binary, test_size=0.2, random_state=42, stratify=y_hypertension_binary
)

# 使用相同的分割索引來分割其他目標變數
_, _, y_train_hg, y_test_hg = train_test_split(
    X, y_hyperglycemia_binary, test_size=0.2, random_state=42, stratify=y_hyperglycemia_binary
)
_, _, y_train_dl, y_test_dl = train_test_split(
    X, y_dyslipidemia_binary, test_size=0.2, random_state=42, stratify=y_dyslipidemia_binary
)

print("資料分割完成:")
print(f"訓練集: {X_train.shape[0]} 人")
print(f"測試集: {X_test.shape[0]} 人")
print(f"特徵數: {X_train.shape[1]} 個")

In [ ]:
# 標準化特徵
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("✅ 特徵標準化完成")
print(f"訓練集形狀: {X_train_scaled.shape}")
print(f"測試集形狀: {X_test_scaled.shape}")

## 4. 建立 Baseline 模型 - Logistic Regression

In [ ]:
def train_and_evaluate_model(X_train, X_test, y_train, y_test, model_name, disease_name):
    """訓練模型並評估效能"""
    
    # 建立模型
    if model_name == 'Logistic Regression':
        model = LogisticRegression(random_state=42, max_iter=1000)
    elif model_name == 'Random Forest':
        model = RandomForestClassifier(n_estimators=100, random_state=42)
    
    # 訓練模型
    model.fit(X_train, y_train)
    
    # 預測
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # 計算指標
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred_proba)
    
    # 顯示結果
    print(f"\n{'='*60}")
    print(f"{disease_name} - {model_name}")
    print(f"{'='*60}")
    print(f"準確率 (Accuracy):  {accuracy:.3f}")
    print(f"精確率 (Precision): {precision:.3f}")
    print(f"召回率 (Recall):    {recall:.3f}")
    print(f"F1 分數:            {f1:.3f}")
    print(f"AUC:                {auc:.3f}")
    
    # 混淆矩陣
    cm = confusion_matrix(y_test, y_pred)
    print(f"\n混淆矩陣:")
    print(f"預測 →    正常  患病")
    print(f"實際 ↓")
    print(f"正常      {cm[0,0]:4d}  {cm[0,1]:4d}")
    print(f"患病      {cm[1,0]:4d}  {cm[1,1]:4d}")
    
    return model, {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'auc': auc
    }

In [ ]:
# 訓練高血壓預測模型
print("\n" + "#"*60)
print("#" + " "*20 + "高血壓預測模型" + " "*24 + "#")
print("#"*60)

model_hp, metrics_hp = train_and_evaluate_model(
    X_train_scaled, X_test_scaled, 
    y_train_hp, y_test_hp,
    'Logistic Regression',
    '高血壓 (Hypertension)'
)

In [ ]:
# 訓練高血糖預測模型
print("\n" + "#"*60)
print("#" + " "*20 + "高血糖預測模型" + " "*24 + "#")
print("#"*60)

model_hg, metrics_hg = train_and_evaluate_model(
    X_train_scaled, X_test_scaled,
    y_train_hg, y_test_hg,
    'Logistic Regression',
    '高血糖 (Hyperglycemia)'
)

In [ ]:
# 訓練高血脂預測模型
print("\n" + "#"*60)
print("#" + " "*20 + "高血脂預測模型" + " "*24 + "#")
print("#"*60)

model_dl, metrics_dl = train_and_evaluate_model(
    X_train_scaled, X_test_scaled,
    y_train_dl, y_test_dl,
    'Logistic Regression',
    '高血脂 (Dyslipidemia)'
)

## 5. 比較不同特徵組合的效果

In [ ]:
# 定義不同的特徵組合
feature_sets = {
    '只用 T1 特徵': demographic_features + t1_features,
    'T1 + Δ1 特徵': demographic_features + t1_features + delta1_features,
    '完整特徵 (T1+T2+Δ1)': all_features
}

# 儲存結果
results = {}

print("\n" + "="*80)
print("比較不同特徵組合 - 高血壓預測")
print("="*80)

for name, features in feature_sets.items():
    # 準備資料
    X_subset = df[features]
    X_train_subset, X_test_subset, y_train, y_test = train_test_split(
        X_subset, y_hypertension_binary, test_size=0.2, random_state=42, stratify=y_hypertension_binary
    )
    
    # 標準化
    scaler = StandardScaler()
    X_train_subset = scaler.fit_transform(X_train_subset)
    X_test_subset = scaler.transform(X_test_subset)
    
    # 訓練模型
    model = LogisticRegression(random_state=42, max_iter=1000)
    model.fit(X_train_subset, y_train)
    
    # 預測和評估
    y_pred = model.predict(X_test_subset)
    y_pred_proba = model.predict_proba(X_test_subset)[:, 1]
    
    # 計算指標
    auc = roc_auc_score(y_test, y_pred_proba)
    f1 = f1_score(y_test, y_pred)
    
    results[name] = {'AUC': auc, 'F1': f1}
    
    print(f"\n{name} ({len(features)} 個特徵):")
    print(f"  AUC: {auc:.3f}")
    print(f"  F1:  {f1:.3f}")

## 6. 視覺化模型效能比較

In [ ]:
# 整合三個疾病的預測結果
all_metrics = pd.DataFrame({
    '高血壓': metrics_hp,
    '高血糖': metrics_hg,
    '高血脂': metrics_dl
}).T

# 繪製比較圖
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# 圖1: 各項指標比較
all_metrics[['accuracy', 'precision', 'recall', 'f1']].plot(kind='bar', ax=axes[0])
axes[0].set_title('三高預測模型效能比較', fontsize=14, fontweight='bold')
axes[0].set_xlabel('疾病類型')
axes[0].set_ylabel('分數')
axes[0].set_ylim([0, 1])
axes[0].legend(['準確率', '精確率', '召回率', 'F1分數'])
axes[0].grid(axis='y', alpha=0.3)
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=0)

# 圖2: AUC 比較
all_metrics['auc'].plot(kind='bar', ax=axes[1], color=['#1f77b4', '#ff7f0e', '#2ca02c'])
axes[1].set_title('AUC 分數比較', fontsize=14, fontweight='bold')
axes[1].set_xlabel('疾病類型')
axes[1].set_ylabel('AUC')
axes[1].set_ylim([0.5, 1])
axes[1].grid(axis='y', alpha=0.3)
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=0)

# 在柱狀圖上加上數值標籤
for i, v in enumerate(all_metrics['auc']):
    axes[1].text(i, v + 0.01, f'{v:.3f}', ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('../../docs/experiments/model_performance_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ 圖表已儲存至: docs/experiments/model_performance_comparison.png")

## 7. 嘗試 Random Forest 模型

In [ ]:
print("\n" + "#"*60)
print("#" + " "*15 + "Random Forest 模型測試" + " "*19 + "#")
print("#"*60)

# 使用 Random Forest 預測高血壓
rf_model_hp, rf_metrics_hp = train_and_evaluate_model(
    X_train_scaled, X_test_scaled,
    y_train_hp, y_test_hp,
    'Random Forest',
    '高血壓 (Hypertension)'
)

# 比較 Logistic Regression vs Random Forest
print("\n" + "="*60)
print("模型比較總結 - 高血壓預測")
print("="*60)
print(f"\nLogistic Regression:")
print(f"  AUC: {metrics_hp['auc']:.3f}")
print(f"  F1:  {metrics_hp['f1']:.3f}")
print(f"\nRandom Forest:")
print(f"  AUC: {rf_metrics_hp['auc']:.3f}")
print(f"  F1:  {rf_metrics_hp['f1']:.3f}")
print(f"\n改善幅度:")
print(f"  AUC: {(rf_metrics_hp['auc'] - metrics_hp['auc'])*100:+.1f}%")
print(f"  F1:  {(rf_metrics_hp['f1'] - metrics_hp['f1'])*100:+.1f}%")

## 8. 特徵重要性分析

In [ ]:
# 取得 Random Forest 的特徵重要性
feature_importance = pd.DataFrame({
    'feature': all_features,
    'importance': rf_model_hp.feature_importances_
}).sort_values('importance', ascending=False)

# 顯示前 15 個最重要的特徵
print("="*60)
print("特徵重要性排名 (Top 15)")
print("="*60)
for i, row in feature_importance.head(15).iterrows():
    print(f"{row['feature']:20s} {row['importance']:.4f}")

# 視覺化
plt.figure(figsize=(10, 6))
top_features = feature_importance.head(15)
plt.barh(range(len(top_features)), top_features['importance'], color='skyblue')
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('重要性分數')
plt.title('特徵重要性排名 - 高血壓預測', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig('../../docs/experiments/feature_importance.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ 圖表已儲存至: docs/experiments/feature_importance.png")

## 9. 總結

### 🎯 完成的工作
1. ✅ 成功建立了三高（高血壓、高血糖、高血脂）的預測模型
2. ✅ 使用 T1 + T2 的資料預測 T3 時的疾病狀態
3. ✅ 比較了 Logistic Regression 和 Random Forest 兩種模型
4. ✅ 分析了特徵重要性

### 📊 模型效能
- **Logistic Regression**: 提供了穩定的 baseline 結果
- **Random Forest**: 通常能獲得更好的預測效能
- **最重要的特徵**: 通常是 T2 時間點的測量值和 Δ 變化量

### 💡 下一步建議
1. **嘗試 XGBoost**: 通常能獲得最佳效能
2. **超參數調優**: 使用 GridSearchCV 或 RandomizedSearchCV
3. **特徵工程**: 創建交互特徵或多項式特徵
4. **集成學習**: 結合多個模型的預測結果
5. **時序分析**: 考慮使用 LSTM 等時序模型